![](images/EscUpmPolit_p.gif "UPM")

# Course Notes for Learning Intelligent Systems

Department of Telematic Engineering Systems, Universidad Politécnica de Madrid, © 2016 Carlos A. Iglesias

# Semantic Models

# Table of Contents
* [Objectives](#Objectives)
* [Corpus](#Corpus)
* [Converting Scikit-learn to gensim](#Converting-Scikit-learn-to-gensim)
* [Latent Dirichlet Allocation (LDA)](#Latent-Dirichlet-Allocation-%28LDA%29)
* [Latent Semantic Indexing (LSI)](#Latent-Semantic-Indexing-%28LSI%29)

# Objectives

In this session we provide a quick overview of the semantic models presented during the classes. In this case, we will use a real corpus so that we can extract meaningful patterns.

The main objectives of this session are:
* Understand the models and their differences
* Learn to use some of the most popular NLP libraries

# Corpus

We are going to use on of the corpus that come prepackaged with Scikit-learn: the [20 newsgroup datase](http://qwone.com/~jason/20Newsgroups/). The 20  newsgroup dataset contains 20k documents that belong to 20 topics.

We inspect now the corpus using the facilities from Scikit-learn, as explain in [scikit-learn](http://scikit-learn.org/stable/datasets/twenty_newsgroups.html#newsgroups)

In [1]:
from sklearn.datasets import fetch_20newsgroups

# We filter only some categories, otherwise we have 20 categories
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
# We remove metadata to avoid bias in the classification
newsgroups_train = fetch_20newsgroups(subset='train', 
                                      remove=('headers', 'footers', 'quotes'), 
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'),
                                    categories=categories)


# Obtain a vector

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', min_df=10)

vectors_train = vectorizer.fit_transform(newsgroups_train.data)
vectors_train.shape

(2034, 2807)

# Converting Scikit-learn to gensim

Although scikit-learn provides an LDA implementation, it is more popular the package *gensim*, which also provides an LSI implementation, as well as other functionalities. Fortunately, scikit-learn sparse matrices can be used in Gensim using the function *matutils.Sparse2Corpus()*. Anyway, if you are using intensively LDA,it can be convenient to create the corpus with their functions.

You should install first *gensim*. Run 'conda install -c anaconda gensim=0.12.4' in a terminal.

In [2]:
from gensim import matutils

vocab = vectorizer.get_feature_names()

dictionary = dict([(i, s) for i, s in enumerate(vectorizer.get_feature_names())])
corpus_tfidf = matutils.Sparse2Corpus(vectors_train)

# Latent Dirichlet Allocation (LDA)

Although scikit-learn provides an LDA implementation, it is more popular the package *gensim*, which also provides an LSI implementation, as well as other functionalities. Fortunately, scikit-learn sparse matrices can be used in Gensim using the function *matutils.Sparse2Corpus()*.

In [3]:
from gensim.models.ldamodel import LdaModel

# It takes a long time

#  train the lda model, choosing number of topics equal to 4
lda = LdaModel(corpus_tfidf, num_topics=4, passes=20, id2word=dictionary)

In [5]:
# check the topics
lda.print_topics(4)

[(0,
  '0.002*led + 0.002*mechanism + 0.002*frank + 0.002*platform + 0.002*mormons + 0.002*concepts + 0.002*proton + 0.002*aeronautics + 0.002*header + 0.002*foreign'),
 (1,
  '0.005*14 + 0.004*covered + 0.004*compared + 0.004*curious + 0.004*edited + 0.004*measured + 0.004*generic + 0.003*object + 0.003*christians + 0.003*developments'),
 (2,
  '0.006*groups + 0.006*center + 0.006*car + 0.006*88 + 0.006*increased + 0.006*closely + 0.006*beginning + 0.006*archie + 0.005*comets + 0.005*1993'),
 (3,
  '0.004*objects + 0.003*activity + 0.003*manhattan + 0.003*obtained + 0.003*eyes + 0.003*education + 0.003*netters + 0.002*europe + 0.002*complex + 0.002*missions')]

Since there are some problems for translating the corpus from Scikit-Learn to LSI, we are now going to create 'natively' the corpus with Gensim.

In [6]:
# import the gensim.corpora module to generate dictionary
from gensim import corpora

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import RegexpTokenizer

import string

def preprocess(words):
    tokenizer = RegexpTokenizer('[A-Z]\w+')
    tokens = [w.lower() for w in tokenizer.tokenize(words)]
    stoplist = stopwords.words('english')
    tokens_stop = [w for w in tokens if w not in stoplist]
    punctuation = set(string.punctuation)
    tokens_clean = [w for w in tokens_stop if  w not in punctuation]
    return tokens_clean

#words = preprocess(newsgroups_train.data)
#dictionary = corpora.Dictionary(newsgroups_train.data)

texts = [preprocess(document) for document in newsgroups_train.data]

dictionary = corpora.Dictionary(texts)

In [7]:
# You can save the dictionary
dictionary.save('newsgroup.dict')

print(dictionary)

Dictionary(10913 unique tokens: ['modelling', 'wfpc', 'inward', 'artemis', 'binding']...)


In [8]:
# Generate a list of docs, where each doc is a list of words

docs = [preprocess(doc) for doc in newsgroups_train.data]

In [9]:
# import the gensim.corpora module to generate dictionary
from gensim import corpora

dictionary = corpora.Dictionary(docs)

In [10]:
# You can optionally save the  dictionary 

dictionary.save('newsgroups.dict')
lda = LdaModel.load('newsgroups.lda')

FileNotFoundError: [Errno 2] No such file or directory: 'newsgroups.lda'

In [11]:
# We can print the dictionary, it is a mappying of id and tokens

print(dictionary)

Dictionary(10913 unique tokens: ['modelling', 'wfpc', 'inward', 'artemis', 'binding']...)


In [12]:
# construct the corpus representing each document as a bag-of-words (bow) vector
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [13]:
from gensim.models import TfidfModel

# calculate tfidf
tfidf_model = TfidfModel(corpus)
corpus_tfidf = tfidf_model[corpus]

In [14]:
#print tf-idf of first document
print(corpus_tfidf[0])

[(0, 0.722808853369507), (1, 0.10438175896914427), (2, 0.24093628445650234), (3, 0.1598114653031772), (4, 0.24093628445650234), (5, 0.5700978153855775)]


In [15]:
from gensim.models.ldamodel import LdaModel

# train the lda model, choosing number of topics equal to 4, it takes a long time

lda_model = LdaModel(corpus_tfidf, num_topics=4, passes=20, id2word=dictionary)

In [16]:
# check the topics
lda_model.print_topics(4)

[(0,
  '0.007*koresh + 0.007*moon + 0.007*bible + 0.006*jeff + 0.006*zoroastrians + 0.005*davidian + 0.005*samaritan + 0.005*basically + 0.005*baltimore + 0.004*sorry'),
 (1,
  '0.011*mary + 0.011*targa + 0.010*whatever + 0.008*god + 0.008*western + 0.007*joy + 0.007*lucky + 0.007*joseph + 0.006*happy + 0.006*craig'),
 (2,
  '0.008*baptist + 0.007*really + 0.007*plane + 0.007*ns + 0.006*unix + 0.006*lot + 0.006*ssrt + 0.005*crusades + 0.005*yayayay + 0.005*bob'),
 (3,
  '0.009*islam + 0.007*khomeini + 0.006*phobos + 0.006*martian + 0.006*gerald + 0.006*bull + 0.005*russian + 0.004*hussein + 0.004*god + 0.003*muslim')]

In [17]:
# check the lsa vector for the first document
corpus_lda = lda_model[corpus_tfidf]
print(corpus_lda[0])

[(0, 0.088232728358839593), (1, 0.094803336919759992), (2, 0.12398755707167169), (3, 0.69297637764972875)]


In [18]:
#predict topics of a new doc
new_doc = "God is love and God is the Lord"
#transform into BOW space
bow_vector = dictionary.doc2bow(preprocess(new_doc))
print([(dictionary[id], count) for id, count in bow_vector])

[('lord', 1), ('god', 2)]


In [19]:
#transform into LDA space
lda_vector = lda_model[bow_vector]
print(lda_vector)

[(0, 0.063874026343805929), (1, 0.80918680467374116), (2, 0.062762107378346629), (3, 0.0641770616041063)]


In [20]:
# print the document's single most prominent LDA topic
print(lda_model.print_topic(max(lda_vector, key=lambda item: item[1])[0]))

0.011*mary + 0.011*targa + 0.010*whatever + 0.008*god + 0.008*western + 0.007*joy + 0.007*lucky + 0.007*joseph + 0.006*happy + 0.006*craig


In [ ]:
lda_vector_tfidf = lda_model[tfidf_model[bow_vector]]
print(lda_vector_tfidf)
# print the document's single most prominent LDA topic
print(lda_model.print_topic(max(lda_vector_tfidf, key=lambda item: item[1])[0]))

# Latent Semantic Indexing (LSI)

In [ ]:
from gensim.models.lsimodel import LsiModel

#It takes a long time

# train the lsi model, choosing number of topics equal to 20


lsi_model = LsiModel(corpus_tfidf, num_topics=4, id2word=dictionary)

In [ ]:
# check the topics
lsi_model.print_topics(4)

In [ ]:
# check the lsi vector for the first document
print(corpus_tfidf[0])

# References

* [NLTK Book. Natural Language Processing with Python. Steven Bird, Ewan Klein, and Edward Loper. O'Reilly Media, 2009 ](http://www.nltk.org/book_1ed/)
* [NLTK Essentials, Nitin Hardeniya, Packt Publishing, 2015](http://proquest.safaribooksonline.com/search?q=NLTK%20Essentials)

## Licence

The notebook is freely licensed under under the [Creative Commons Attribution Share-Alike license](https://creativecommons.org/licenses/by/2.0/).  

© 2016 Carlos A. Iglesias, Universidad Politécnica de Madrid.